### Basic POST setup

In [25]:
from fastapi import FastAPI
from pydantic import BaseModel

# Initialize the FastAPI app
app = FastAPI()

# Define the Pydantic model
class Query(BaseModel):
    query: str # type hint, query must be of type str

# This defines a POST endpoint /chat/.
# When this endpoint is called, the chat function will be triggered.
@app.post("/chat/")
async def chat(query: Query):
    # query is automatically validated by FastAPI
    # using the pydantic model
    return {"received_query": query.query}

#### Testing the Query object

In [26]:
test_query = Query(query="hello, testing")
print(test_query)
print(test_query.query)

query='hello, testing'
hello, testing


In [27]:
test_query = Query(query=1234)
print(test_query)
print(test_query.query)

ValidationError: 1 validation error for Query
query
  Input should be a valid string [type=string_type, input_value=1234, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type

#### Simulate sending JSON data using a test client 

In [ ]:
from fastapi.testclient import TestClient

# Use the app defined above
client = TestClient(app)

test_data = {
    "query": "hello, how are you?"
}

response = client.post("/chat/", json=test_data)
print(response.json()) 

{'received_query': 'hello, how are you?'}


### Basic Websocket setup

In [28]:
from fastapi import WebSocket

app = FastAPI()

@app.websocket("/ws/")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()  # Accept the WebSocket connection
    await websocket.send_text("Welcome to the WebSocket chat!")  # Initial message to the client

    try:
        while True:
            # Wait to receive a message from the client
            client_message = await websocket.receive_text()
            
            # Process the message (this is where you can add any logic)
            response_message = f"Message received: {client_message}"
            
            # Send a response back to the client
            await websocket.send_text(response_message)
    except Exception as e:
        # Handle errors and send them to the client
        await websocket.send_text(f"Error: {e}")
    finally:
        # Cleanly close the WebSocket connection
        await websocket.close()

In [29]:
client = TestClient(app)

def test_websocket():

    with client.websocket_connect("/ws/") as websocket:

        # Receive the initial server message
        response = websocket.receive_text()
        print(response)

        # Send a message and receive the server's response
        websocket.send_text("hello websocket!")
        response = websocket.receive_text()
        print(response)

        # Send another message and receive the server's response
        websocket.send_text("respond please")
        response = websocket.receive_text()
        print(response)

test_websocket()

Welcome to the WebSocket chat!
Message received: hello websocket!
Message received: respond please
